# Alunos Regulares IA-024-2024S1 FEEC-UNICAMP
versão 26 de fevereiro de 2024, 19h

## Instalação e importação de pacotes

In [1]:
#!pip install torchtext
#!pip install 'portalocker>=2.0.0'

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchtext.datasets import IMDB
from collections import Counter
import torch.nn as nn
import torch.optim as optim
import string

## I - Vocabulário e Tokenização

In [3]:
# limit the vocabulary size to 20000 most frequent tokens
vocab_size = 20000

negative = 0
positive = 0
average_text_length = 0
train_set = []

counter = Counter()
for (target, line) in list(IMDB(split='train')):
    
    # Counting how many positive or negative analysis sample and ]
    average_text_length = len(line.split()) + average_text_length
    if target == 1:
        negative += 1
    elif target == 2:
        positive += 1
    line = line.lower().translate(str.maketrans('', '', string.punctuation))
    
    counter.update(line.split())
    train_set.append(line)

# create a vocabulary of the 20000 most frequent tokens
most_frequent_words = sorted(counter, key=counter.get, reverse=True)[:vocab_size]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}
vocab_size = len(vocab)

print(f'{positive} são positivas, {negative} são negativas e existe um total de {positive + negative} amostras.')
print(f'O comprimento é médio de palavras por linha é de {average_text_length/len(list(IMDB(split='train')))} palavras.')
print(f'As cinco palavras mais frequentes são {most_frequent_words[:5]}')
print(f'As cinco palavras menos frequentes são {most_frequent_words[-1:-6: -1]}')

12500 são positivas, 12500 são negativas e existe um total de 25000 amostras.
O comprimento é médio de palavras por linha é de 233.7872 palavras.
As cinco palavras mais frequentes são ['the', 'and', 'a', 'of', 'to']
As cinco palavras menos frequentes são ['goring', 'cacoyannis', 'showings', 'dolores', 'ponderosa']


In [4]:
unk_words_length = 0

def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in sentence.split()] # 0 for OOV

print(encode_sentence("I like Pizza.", vocab))
print('0 é a codificação que atribui o código de "unknown token" para palavras desconhecidas. Caso da palavra "Pizza", que não está no vocab.')

for sentence in train_set:
    unk_words_length += encode_sentence(sentence, vocab).count(0)

print(f'No dataset de treino, existem {unk_words_length} unknown tokens.')

[0, 38, 0]
0 é a codificação que atribui o código de "unknown token" para palavras desconhecidas. Caso da palavra "Pizza", que não está no vocab.
No dataset de treino, existem 214473 unknown tokens.


In [5]:
quantidade_1 = 0
for i,_ in list(IMDB(split='train'))[:200]:
    if i == 1:
        quantidade_1 += 1
    
print(f'A quantidade de classes 1 no dataset de 200 elementos é {quantidade_1}')
print('A razão pelo qual a precisão se aproxima de 100% é porque o conjunto de treinamento está composto apenas por amostras de uma única classe. Dessa forma, ele decorou como classificar um unico tipo de texto.')

# Deixando o dataset balanceado.
negative_train_data = [i for i in list(IMDB(split='train')) if i[0] == 1][:100]
positive_train_data = [i for i in list(IMDB(split='train')) if i[0] == 2][:100]
balanced_train_data = negative_train_data + positive_train_data

quantidade_1 = 0
for i,_ in balanced_train_data:
    if i == 1:
        quantidade_1 += 1
print(f'Após o balanceamento, a quantidade de classes 1 no dataset de 200 elementos é {quantidade_1}')

A quantidade de classes 1 no dataset de 200 elementos é 200
A razão pelo qual a precisão se aproxima de 100% é porque o conjunto de treinamento está composto apenas por amostras de uma única classe. Dessa forma, ele decorou como classificar um unico tipo de texto.
Após o balanceamento, a quantidade de classes 1 no dataset de 200 elementos é 100


## II - Dataset

In [6]:
# Classe não otimizada

"""from torch.nn.functional import one_hot
# Dataset Class with One-hot Encoding
class IMDBDataset(Dataset):
    def __init__(self, split, vocab):
        self.data = list(IMDB(split=split))
        self.vocab = vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        target, line = self.data[idx]
        target = 1 if target == 1 else 0

        # one-hot encoding
        X = torch.zeros(len(self.vocab) + 1)
        for word in encode_sentence(line, self.vocab):
            X[word] = 1

        return X, torch.tensor(target)

# Load Data with One-hot Encoding
train_data = IMDBDataset('train', vocab)
test_data = IMDBDataset('test', vocab)
"""

"from torch.nn.functional import one_hot\n# Dataset Class with One-hot Encoding\nclass IMDBDataset(Dataset):\n    def __init__(self, split, vocab):\n        self.data = list(IMDB(split=split))\n        self.vocab = vocab\n\n    def __len__(self):\n        return len(self.data)\n\n    def __getitem__(self, idx):\n        target, line = self.data[idx]\n        target = 1 if target == 1 else 0\n\n        # one-hot encoding\n        X = torch.zeros(len(self.vocab) + 1)\n        for word in encode_sentence(line, self.vocab):\n            X[word] = 1\n\n        return X, torch.tensor(target)\n\n# Load Data with One-hot Encoding\ntrain_data = IMDBDataset('train', vocab)\ntest_data = IMDBDataset('test', vocab)\n"

In [7]:
class IMDBDataset(Dataset):
    def __init__(self, split, vocab):
        self.data = list(IMDB(split=split))
        self.vocab = vocab
        self.one_hot_encoded_data = self.preprocess_data()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.one_hot_encoded_data[idx]

    def preprocess_data(self):
        one_hot_encoded_data = []
        for target, line in self.data:
            target = 1 if target == 1 else 0
            line = line.lower().translate(str.maketrans('', '', string.punctuation))
            # One-hot encoding
            X = torch.zeros(len(self.vocab) + 1)
            for word in encode_sentence(line, self.vocab):
                X[word] = 1
            one_hot_encoded_data.append((X, torch.tensor(target)))
        return one_hot_encoded_data

# Load Data with One-hot Encoding
train_data = IMDBDataset('train', vocab)
test_data = IMDBDataset('test', vocab)


In [8]:
# Contadores para amostras positivas e negativas
positive_samples = 0
negative_samples = 0

# Iterar sobre o conjunto de dados de treinamento
for target, line in train_data.data:
    # Incrementar o contador correspondente à classe da amostra
    if target == 1:
        positive_samples += 1
    elif target == 2:
        negative_samples += 1
print("Número de amostras positivas, ao chamar o atributo do objeto (train_data.data):", positive_samples)
print("Número de amostras negativas, ao chamar o atributo do objeto (train_data.data):", negative_samples)

negative_samples = 0
positive_samples = 0

for i in train_data:
    if i[1] == 1:
        negative_samples += 1
    elif i[1] == 0:
        positive_samples += 1
    
print("Número de amostras positivas, ao chamar apenas o objeto (train_data):", positive_samples)
print("Número de amostras negativas, ao chamar apenas o objeto (train_data):", negative_samples)

"""Calculando o comprimento médio das palavras codificadas"""

# Calculate the number of non-zero elements in each one-hot encoded vector


counts = torch.tensor([torch.sum(X != 0).item() for X, _ in train_data])

# Calculate the average number of words encoded in each one-hot vector
average_non_zero_elements = torch.mean(counts.float()).item()

print(f'O número médio de palavras codificadas em cada vetor one-hot é de: {average_non_zero_elements}')
print("Comprimento médio de cada texto (em palavras):", average_text_length / len(train_data))
print('Existe a diferença da dimensão do comprimento médio entre o texto pois o onehot considera apenas as palavras que estão no vocabulário.')

Número de amostras positivas, ao chamar o atributo do objeto (train_data.data): 12500
Número de amostras negativas, ao chamar o atributo do objeto (train_data.data): 12500
Número de amostras positivas, ao chamar apenas o objeto (train_data): 12500
Número de amostras negativas, ao chamar apenas o objeto (train_data): 12500
O número médio de palavras codificadas em cada vetor one-hot é de: 133.73216247558594
Comprimento médio de cada texto (em palavras): 233.7872
Existe a diferença da dimensão do comprimento médio entre o texto pois o onehot considera apenas as palavras que estão no vocabulário.


## III - Data Loader

In [9]:
batch_size = 128
# define dataloaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_data,  batch_size=batch_size, shuffle=False)

In [10]:
# Número total de iterações no train_loader
print("Número total de iterações no train_loader:", len(train_loader))

# Imprimindo o número de amostras do último batch
print(f"Número de amostras do último batch: {len(train_loader.dataset) % batch_size}")

# Calculando a relação R para cada batch usando list comprehension
R_values = [torch.sum(targets).item() / len(targets) for inputs, targets in train_loader]

# Calculando a média dos valores de R
average_R = sum(R_values) / len(R_values)
print("Média dos valores de R:", average_R)

# Calculando o valor médio de R usando tensores do PyTorch
mean_R = torch.mean(torch.tensor(R_values))
print("Valor médio de R:", mean_R.item())

# Lista de valores de R para cada batch
print("Valores de R para cada batch:", R_values)

# Obtendo um batch do train_loader
batch = next(iter(train_loader))

# Extraindo os elementos do batch
inputs, targets = batch

# Mostrando a estrutura do batch
print("Shape dos inputs:", inputs.shape)
print("Tipo de dado dos inputs:", inputs.dtype)
print("Shape dos targets:", targets.shape)
print("Tipo de dado dos targets:", targets.dtype)

Número total de iterações no train_loader: 196
Número de amostras do último batch: 40
Média dos valores de R: 0.5001753826530612
Valor médio de R: 0.500175416469574
Valores de R para cada batch: [0.4609375, 0.5625, 0.484375, 0.53125, 0.515625, 0.46875, 0.4375, 0.5703125, 0.5078125, 0.53125, 0.5, 0.515625, 0.5546875, 0.421875, 0.5078125, 0.5, 0.5234375, 0.546875, 0.5, 0.5234375, 0.4765625, 0.484375, 0.6015625, 0.4921875, 0.4765625, 0.40625, 0.484375, 0.5234375, 0.5, 0.5546875, 0.546875, 0.453125, 0.53125, 0.5, 0.4375, 0.59375, 0.5078125, 0.578125, 0.5, 0.53125, 0.5703125, 0.5546875, 0.546875, 0.4375, 0.515625, 0.5625, 0.5, 0.5390625, 0.453125, 0.515625, 0.4765625, 0.4921875, 0.53125, 0.5703125, 0.484375, 0.515625, 0.4921875, 0.5078125, 0.546875, 0.484375, 0.515625, 0.4921875, 0.4921875, 0.5, 0.53125, 0.4453125, 0.5078125, 0.53125, 0.4375, 0.5234375, 0.5, 0.515625, 0.5234375, 0.4921875, 0.5, 0.5, 0.5234375, 0.4375, 0.5078125, 0.46875, 0.4375, 0.3828125, 0.5390625, 0.4765625, 0.5, 0.48437

## IV - Modelo

In [11]:
class OneHotMLP(nn.Module):
    def __init__(self, vocab_size):
        super(OneHotMLP, self).__init__()

        self.fc1 = nn.Linear(vocab_size+1, 200)
        self.fc2 = nn.Linear(200, 1)

        self.relu = nn.ReLU()

    def forward(self, x):
        o = self.fc1(x.float())
        o = self.relu(o)
        return self.fc2(o)

# Model instantiation
model = OneHotMLP(vocab_size)

In [12]:
inputs,targets = next(iter(train_loader))
logits = model(inputs)
probabilities = torch.sigmoid(logits)

# Imprimir as probabilidades
for probability in probabilities:
    print(f'Probabilidade: aleatoria {probability.item():.4f}')
    break
    
# Calcular a Loss utilizando a função de entropia cruzada
loss = - (targets * torch.log(probabilities) + (1 - targets) * torch.log(1 - probabilities))
total_loss = torch.mean(loss)
print("Valor da Loss:", total_loss.item())

# Instanciar a função de Loss BCELoss e BCEWithLogitsLoss
bce_loss_function = nn.BCELoss()
bce_with_logits_loss_function = nn.BCEWithLogitsLoss()


# Expandir as dimensões do alvo para corresponder à dimensão da entrada
targets_expanded = targets.unsqueeze(1).float()

# Calcular a Loss utilizando a função de BCELoss
loss_bce = bce_loss_function(probabilities, targets_expanded)

print("Valor da Loss utilizando BCELoss:", loss_bce.item())

# Calcular a Loss utilizando a função de BCEWithLogitsLoss
loss_bce_with_logits = bce_with_logits_loss_function(logits.squeeze(), targets.float())

print("Valor da Loss utilizando BCEWithLogitsLoss:", loss_bce_with_logits.item())


Probabilidade: aleatoria 0.5125
Valor da Loss: 0.6891483664512634
Valor da Loss utilizando BCELoss: 0.6896375417709351
Valor da Loss utilizando BCEWithLogitsLoss: 0.6896375417709351


In [ ]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print('GPU:', torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print('using CPU')

correct = 0
total = 0

# Defina o modelo para o modo de avaliação
model = OneHotMLP(vocab_size)
model = model.to(device)

# Extraia o primeiro batch dos dados de treinamento
inputs, targets = next(iter(train_loader))
inputs = inputs.to(device)
targets = targets.to(device)

# Faça a predição para cada amostra no batch
with torch.no_grad():
    logits = model(inputs)
    probabilities = torch.sigmoid(logits)
    predicted_classes = (probabilities > 0.5).float()

# Compare as classes previstas com os targets esperados para calcular a acurácia
correct += (predicted_classes == targets.unsqueeze(1)).sum().item()
total += targets.size(0)

# Calcule a acurácia
accuracy = correct / total

print(f'Acurácia do primeiro batch: {accuracy * 100:.2f}%')

predicted = torch.round(torch.sigmoid(logits.squeeze()))
total = targets.size(0)
correct = (predicted == targets).sum().item()
print(100*correct/total)


In [ ]:
# Defina o modelo para o modo de avaliação
model = OneHotMLP(vocab_size)
model = model.to(device)

# Extraia o primeiro batch dos dados de treinamento
inputs, targets = next(iter(train_loader))
inputs = inputs.to(device)
targets = targets.to(device)


# Calcular as previsões do modelo para o primeiro batch
logits = model(inputs)
probabilities = torch.sigmoid(logits)

# Calcular a Loss utilizando a função de entropia cruzada
loss = - (targets * torch.log(probabilities) + (1 - targets) * torch.log(1 - probabilities))
total_loss = torch.mean(loss)

print("Valor da Loss:", total_loss.item())


In [ ]:
# Tamanho dos pesos e biases da camada fc1
size_fc1_weight = model.fc1.weight.numel()
size_fc1_bias = model.fc1.bias.numel()

# Tamanho dos pesos e biases da camada fc2
size_fc2_weight = model.fc2.weight.numel()
size_fc2_bias = model.fc2.bias.numel()

# Total de parâmetros do modelo
total_params_fc1 = size_fc1_weight + size_fc1_bias
total_params_fc2 = size_fc2_weight + size_fc2_bias
total_params = total_params_fc1 + total_params_fc2

print(f'Tamanho dos pesos da camada fc1: {size_fc1_weight}, Bias: {size_fc1_bias}')
print(f'Tamanho dos pesos da camada fc2: {size_fc2_weight}, Bias: {size_fc2_bias}')
print(f'Total de parâmetros do modelo: {total_params}')


## V - Laço de Treinamento - Otimização da função de Perda pelo Gradiente descendente

In [ ]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print('GPU:', torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print('using CPU')

In [ ]:
import time

model = OneHotMLP(vocab_size)

model = model.to(device)
# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.18)
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    start_time = time.time()  # Start time of the epoch
    model.train()
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device) 
        # Forward pass
        logits = model(inputs)
        loss = criterion(logits.squeeze(), targets.float())
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    end_time = time.time()  # End time of the epoch
    epoch_duration = end_time - start_time  # Duration of epoch

    print(f'Epoch [{epoch+1}/{num_epochs}], \
            Loss: {loss.item():.4f}, \
            Elapsed Time: {epoch_duration:.2f} sec')

## VI - Avaliação

In [ ]:
## evaluation
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        logits = model(inputs)
        predicted = torch.round(torch.sigmoid(logits.squeeze()))
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    print(f'Test Accuracy: {100 * correct / total}%')

#### Calculating the gpu time

In [ ]:
import time

model = model.to(device)
# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Variáveis para armazenar os tempos
tempo_forward_total = 0
tempo_backward_total = 0
tempo_total_loop = 0

# Contador para controlar os dois loops
cont_loop = 0

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    start_time_loop = time.time()  # Hora de início do loop
    model.train()
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        # Início do tempo do forward
        start_time_forward = time.time()
        # Forward pass
        logits = model(inputs)
        loss = criterion(logits.squeeze(), targets.float())
        # Fim do tempo do forward
        end_time_forward = time.time()
        tempo_forward = end_time_forward - start_time_forward
        tempo_forward_total += tempo_forward
        
        # Início do tempo do backward
        start_time_backward = time.time()
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Fim do tempo do backward
        end_time_backward = time.time()
        tempo_backward = end_time_backward - start_time_backward
        tempo_backward_total += tempo_backward
        
        # Contador do loop
        cont_loop += 1
        
        # Se o contador for igual a 2, saia do loop
        if cont_loop == 2:
            break
    
    # Fim do tempo do loop
    end_time_loop = time.time()
    tempo_total = end_time_loop - start_time_loop
    tempo_total_loop += tempo_total
    
    # Print dos tempos
    print(f'Tempo do laço: {tempo_total:.3f} segundos')
    print(f'Tempo do forward: {tempo_forward_total:.3f} segundos')
    print(f'Tempo do backward: {tempo_backward_total:.3f} segundos')

# Calculando a média dos tempos para cada iteração
media_tempo_loop = tempo_total_loop / num_epochs
media_tempo_forward = tempo_forward_total / num_epochs
media_tempo_backward = tempo_backward_total / num_epochs

print(f'Média do tempo do laço: {media_tempo_loop:.3f} segundos')
print(f'Média do tempo do forward: {media_tempo_forward:.3f} segundos')
print(f'Média do tempo do backward: {media_tempo_backward:.3f} segundos')
